In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import os
import sys 
import json
import glob
import random
import collections
import time
import re
import math
import numpy as np
import pandas as pd
import cv2

We will be following the code presented to us in Keras notebook for 3D image classification on CT scan of chest. Here is the code one can find : https://keras.io/examples/vision/3D_image_classification/

In [ ]:
# Lets read the train data and the train labels. 
train_data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
IMAGE_SIZE = 128
NUM_IMAGES = 64
def load_dicom_image(path, img_size=IMAGE_SIZE, voi_lut=True, rotate=0):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    if rotate > 0:
        rot_choices = [0, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
        data = cv2.rotate(data, rot_choices[rotate])
        
    data = cv2.resize(data, (img_size, img_size))
    return data


def load_dicom_images_3d(scan_id, num_imgs=NUM_IMAGES, img_size=IMAGE_SIZE, mri_type="FLAIR", split="test", rotate=0):

    files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f, rotate=rotate) for f in files[p1:p2]]).T 
    if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)
        
    if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)
            
    return np.expand_dims(img3d,0)



In [ ]:
a = load_dicom_images_3d("00001")
print(a.shape)
print(np.min(a), np.max(a), np.mean(a), np.median(a))
image = a[0]
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 15]), cmap="gray")

## EXPLANATION OF ABOVE DEF FUNCTIONS 

Lets trace how this image is getting loaded by completely following the def function to have a full understanding of what this 3D image

In [ ]:
num_imgs=NUM_IMAGES
img_size=IMAGE_SIZE
mri_type="FLAIR"
split="test"
rotate=0
scan_id = "00001"

The below address essentially points out that we should take all the files within 00001 under test folder under flair mri images. We are pointing to all the images within that. 

In [ ]:
f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"

This is a simple lambda function that returns the key if all is numerical otherwise just the numerical part. 

In [ ]:
key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)]

In [ ]:
files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
print(files)

In [ ]:
middle = len(files)//2
num_imgs2 = num_imgs//2
p1 = max(0, middle - num_imgs2)
p2 = min(len(files), middle + num_imgs2)

print(p1,p2)

I am choosing 64 images within 72 and 136 i.e my number of Images and stacking them in a numpy array taking their transpose 

In [ ]:
img3d = np.stack([load_dicom_image(f, rotate=rotate) for f in files[p1:p2]]).T 

In [ ]:
img3d.T.shape

In [ ]:
img3d.shape
print(img3d.shape,img3d.shape[-1])

In [ ]:
img3d.shape[-1] < num_imgs

If the sampled images are less than 64 i.e the number of images then we will put black array of pixels of the same size to take the number of images to 64

In [ ]:
if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)

In [ ]:
print(np.min(img3d),np.max(img3d))

The below operation is basically the scaling of pixels where we subtract the minimum value and divide by the maximum value. 

In [ ]:
if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)

In [ ]:
img3d.shape

In [ ]:
np.expand_dims(img3d,0).T.shape

## Generate the training Dataset

In [ ]:
df_train_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df_train_labels.head()

In [ ]:
train_data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
print(min(os.listdir(train_data_directory)), max(os.listdir(train_data_directory)))

Lets not take all 1010 samples for training as this will led to a lot of memory and time consumption hence lets take only 140 samples out of these all. You can use below line of code for all. 

In [ ]:
total_training_scans = np.array([load_dicom_images_3d(a, split = 'train') for a in os.listdir(train_data_directory)])

In [ ]:
count = 0
list_train_files = []
for file in os.listdir(train_data_directory):
    if count<140:
        list_train_files.append(file)
        count = count+1

In [ ]:
len(list_train_files)

In [ ]:
total_training_scans = np.array([load_dicom_images_3d(a, split = 'train') for a in list_train_files])

In [ ]:
print(total_training_scans.shape)
print(len(total_training_scans))
image = total_training_scans[20]
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 25]), cmap="gray")

In [ ]:
# Split data in the ratio 70-30 for training and validation.
x_train = np.array([train.T for train in total_training_scans[:100]])
x_val = np.array([train.T for train in total_training_scans[100:]])


In [ ]:
x_train.shape

In [ ]:
x_train = x_train[:100]

In [ ]:
x_train.shape

In [ ]:
total_training_labels = df_train_labels["MGMT_value"].to_list()

In [ ]:
total_training_labels = total_training_labels[:140]

In [ ]:
len(total_training_labels)

In [ ]:
y_train = total_training_labels[:100]
y_val = total_training_labels[100:]

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()


# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
a = load_dicom_images_3d("00001")
print(a.shape)
print(np.min(a), np.max(a), np.mean(a), np.median(a))
image = a[0]
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 30]), cmap="gray")
plot_slices(4, 10, 128, 128, image[:, :, :40])

## MODEL

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((depth,width, height,1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

Training it for one epoch because we have a limited memory with this. All above transpose operations has been made in order to provide an input image according to the dimensions we have specified. Also We need to ensure that we provide a batch in order to cover for None in (None, 64,128,128,1). Check the stack overflow here for this : https://stackoverflow.com/questions/49840968/valueerror-input-0-is-incompatible-with-layer-conv1d-1-expected-ndim-3-found

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 1
model.fit(
    tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(4),
    validation_data=tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(4),
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)